# 📅 Week 4 -Counterfactual Explanations using Tabular Data
#### 🚨 **First things first! Make a copy of this notebook. Your changes will not save unless you create your own copy!**🚨


You are a data scientist and risk modeler of a new fintech company that provides personal loans to individuals. As a startup, the company has limited staff. For this reason, your company decides to create a machine learning algorithm that can assess risk and categories potential loan takers in to low and high risk category. 

However, the company has a small pool of data from the manual decisions it made. Despite these challenges, your team is determined to develop a state-of-the-art loan default/risk prediction model to differentiate the company from its competitors.

<center><img src = 'https://media1.giphy.com/media/3orif2cYsDAMzFMvjW/giphy.gif?cid=ecf05e47d9yxi8z0ftvw5ig3xytkvdky6oczi3uyrc4qet32&rid=giphy.gif&ct=g'></center>

The stakes are high as there is a risk of losing out on potential customers and damaging the reputation of the company in case of false rejections. Also, you want to inform your customers why exactly were they rejected and suggest suitable actionables they can take to lower their risk and hypothetically get the credit loan. Your team decides that the obvious choice would be to generate counterfactuals using XAI principles and present them to customers. 

Therefore, your job as the data scientist is to build a machine learning model that predicts risk on the manual data. To generate and verify if the counterfactuals are reasonable and are plausible.

#📦 Installation and Imports
We will use same [OmnixAI](https://github.com/salesforce/OmniXAI) python package for generating counterfactuals

In [ ]:
!pip install omnixai

In [ ]:
## The Usual Suspects
import tensorflow as tf
import itertools
import numpy as np
import pandas as pd
from typing import Any
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer


## For visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=14)

## Training pytorch tabular model

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

## OmniXAI Counterfactual Explainer
from omnixai.data.tabular import Tabular
from omnixai.explainers.tabular import CounterfactualExplainer
from omnixai.explainers.tabular.specific.decision_tree import TreeClassifier

## 💻Dataset - German Credit Risk
For developing a machine learning model, you decide to use a [real data](https://archive.ics.uci.edu/ml/datasets/South+German+Credit) and understand the factors that influenced the decision of credit risk. For this project, we will be using the German Credit Risk dataset. Download the dataset from [Kaggle here](https://www.kaggle.com/datasets/kabure/german-credit-data-with-risk) or just run the cell below. 

In [ ]:
## Read CSV file and import to dataframe from the url
url = 'https://drive.google.com/file/d/13vAvup3zgmkPOJ9P4ulRkQ3BQCN7nqe_/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path, index_col=0)
df.head()

#🔎 Exploratory Data Analysis
Let us understand this dataset and make it ready for our ML model. The attributes are pretty much self-explanatory. However, there are some `NaN` in the data. Let us check how many columns have missing information by exploring this dataset

## Data Cleaning

In [ ]:
df.info()

Only categorical variables (Dtype - `object`) have missing entries. For these are categorical variables, we will fill the NaN with an additional category called `other`

In [ ]:
## TODO: For the columns with missing information, fill the NaN with the variable 'other'
# df.fillna(..., inplace = True)
# df.head()

In [ ]:
df.info()

Our dataset now has 1000 rows that are valid with no NaNs. The column `Risk` is the predictor of interest that catergorizes the individual based on different attributes into `good` or `bad`. Let us see the number of people with `good` and `bad` risk profile

In [ ]:
df['Risk'].value_counts().plot(kind ='bar')

### Explore Categorical Variables

#### Convert categorical variables with two classes to binary format
In binary format, we assign a class with label `0` and other class with `1`

In [ ]:
## TO DO: Convert `Risk` and `Sex` to a binary variable
## Use .map() method and convert bad -> 0 and good -> 1 & male ->1 and female ->0

# df['Risk'] = df['Risk'].map(...)
# df['Sex'] = df['Sex'].map(...)

#### Convert categorical variables with >2 classes to ordinal format
In ordinal format, we assign a number for each class starting from 0. The columns are `Job` ,  `Housing` , `Saving accounts` , `Checking account`, and `Purpose`. Let us explore each of them

In [ ]:
cat_variables = ['Job' , 'Housing' , 'Saving accounts' , 'Checking account', 'Purpose']
fig, ax = plt.subplots(nrows=2,ncols=3, figsize= (14,10))
for i,category in enumerate(cat_variables):
  j = i if i < 3 else i % 3
  df[category].value_counts().plot(kind = 'bar', ax = ax[int(i/3),j], title = category)
fig.delaxes(ax[1,2])
fig.tight_layout()

Notice that the `Job` column is already represented in an ordinal format. Let us convert the rest of them too. In order to keep track of the relationship of classe and labels, let us store the class dictionaries in `class_to_labels` variable all categories

In [ ]:
class_to_labels = {}
cat_to_ordinal = ['Housing' , 'Saving accounts' , 'Checking account', 'Purpose']
for category in cat_to_ordinal:
    values = df[category].unique()
    ids = range(0,len(values))
    cat_dict = dict(zip(values,ids))
    df[category] = df[category].map(cat_dict)
    class_to_labels[category] = cat_dict

In [ ]:
df

In [ ]:
class_to_labels

## Explore Distributions

In [ ]:
sns.displot(
    df, x="Age", col="Risk", row="Sex",
    binwidth=3, height=5, facet_kws=dict(margin_titles=True)
)

In [ ]:
fig, ax = plt.subplots(figsize= (12,10))
sns.violinplot(data=df, x="Risk", y="Credit amount", hue="Housing", inner="box", linewidth=1,pallette  = 'tab10')
sns.despine(left=True)

## View Correlations

Now, let us see which attributes are correlated (positive/negative) with the `Risk`. To do this we need to convert the `Risk` to binary variable

In [ ]:
## TO DO: Use `seaborn` heatmap to display the correlations. Fill in the `visualize_corr` function
## Make sure to display the annotations and colormap
def visualize_corr(df, figsize = (12,12)):
  plt.figure(figsize=figsize)
  pass


def visualize_corr(df, figsize = (12,12)):
  plt.figure(figsize=figsize)
  sns.heatmap(df.corr(), annot=True)
  plt.show()

In [ ]:
visualize_corr(df)

## 🚨TODO: Let's build some Intuition 🤔

<img src ='https://cdn.streamelements.com/uploads/71a1c318-9fd1-4cf1-b4ea-d090a49cb85c.gif'>

Based on the exploratory data analysis above, list the attributes/features that are most influential the deciding the `Risk` as good/bad.


1.   "List item here"
2.   "List item here"








Answer the following questions
1. List the attributes that make it less risky(good)
2. List the attributes that make it more risky(bad)
3. Did you notice any trends? Do they sound reasonable?


#🤖 Machine Learning Model
Using the manual data as input, let us build our machine learning model. We will build a Neural Network based classifiers in this section. Before we proceed, we need to normalize our numnerical variables and split the dataset into `train` and `test` with a 80:20 split 

In [ ]:
df

In [ ]:
## TODO: Scale numerical variables to [0,1] using `MinMaxScalar()` from sklearn

numeric_columns= ['Age','Credit amount','Duration']
# Scaler = ...
# df[numeric_columns] = ...


In [ ]:
## Note: You can convert to the original value using the `inverse_transform`
Scaler.inverse_transform(df.head()[numeric_columns])

In [ ]:
## TODO: Split the data into train and test. Initialize variables X & Y and pass into `train_test_split` function
## Make sure to fill the remaining blanks and use a random_state

# X = ...
# Y = ...
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=..., random_state=..., stratify=y)


In [ ]:
### Here is the function to build and train the nueral network model
def train_tf_model(x_train, y_train, x_test, y_test):
    y_train = tf.keras.utils.to_categorical(y_train, 2)
    y_test = tf.keras.utils.to_categorical(y_test, 2)

    model = tf.keras.models.Sequential()
    ### Fill out the inpout size based on the number of input variables
    model.add(tf.keras.layers.Input(shape=(9,)))
    model.add(tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax))

    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.1,
        decay_steps=1,
        decay_rate=0.99,
        staircase=True
    )
    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=0.9,
        nesterov=True
    )
    loss = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=0)
    train_loss, train_accuracy = model.evaluate(x_train, y_train, batch_size=51, verbose=0)
    test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=51, verbose=0)

    print('Train loss: {:.4f}, train accuracy: {:.4f}'.format(train_loss, train_accuracy))
    print('Test loss:  {:.4f}, test accuracy:  {:.4f}'.format(test_loss, test_accuracy))
    return model

In [ ]:
## Build and train the nueral network
model_nn = train_tf_model(x_train, y_train, x_test, y_test)

In [ ]:
model_nn.predict(x_test[:5])

# 🔄️ Generate Counterfactuals
Now that we have our ML model ready, let us generate our CounterfactualExplainer using OmnixAI library. Read this [paper](https://arxiv.org/ftp/arxiv/papers/1711/1711.00399.pdf) for more information on counterfactuals

In [ ]:
## OmnixAI requires the data to be in a Tabular, Image or Text format. Let's convert our Dataframe to a Tabular format
## TODO: Pass the training data and features

# feature_names = ...

## Used for initializing the explainer
# tabular_data_train = Tabular(
#     data = ...,
#     feature_columns=feature_names
# )

# tabular_data_test = Tabular(
#     data = ...,
#     feature_columns=feature_names
# )

In [ ]:
## TODO: Pass the training data and model to the Counterfactual explainer
# explainer_nn = CounterfactualExplainer(training_data=...,
#                                        predict_function=...)


In [ ]:
## Time to generate some counterfactuals!!!
explanations = explainer_nn.explain(tabular_data_test[1])

In [ ]:
explanations.ipython_plot(index = 0)

In [ ]:
## TODO: Sample few records in the test data where the `Risk` is predicted as `bad`(0) and generate a counterfactuals
## NOTE: Select a sample size N if the explainer takes long

# N = ...
# RANDOM_STATE = ...
# tab_indices = y_test[y_test == ...]\
# .sample(n=..., random_state=...)\
# .index\
# .tolist()

N = 1
RANDOM_STATE = 42
tab_indices = y_test[y_test == 0].sample(n=N, random_state=RANDOM_STATE).index.tolist()

In [ ]:
explanations_risk_0 = explainer_nn.explain(tabular_data_test.to_pd().loc[tab_indices])

In [ ]:
explanations_risk_0.ipython_plot()

In [ ]:
## TODO: Find sample records in the train & test data where the model made an incorrect/different prediction and generate counterfactuals for those predictions
## Fill out the missing entries in the function

# def find_incorrect_pred_indices(
#     model: Any, X: pd.DataFrame, y: pd.Series, size: int = N
# ) -> None:
#     """
#     Find the incorrect predictions from a model and generates the counterfactual
#     explanations for the particular dataset that the model was trained/evaluated on.
    
#     Args:
#         model: The model used for training
#         X:     The dataset used on the model training/evaluation excluding the target column
#         y:     The target column values of X
#         size:  The size of the random samples to select from each of the false positives and false negatives.
#                The bigger the size, the longer the computation of the counterfactuals. 
#     """
#     predictions = model.predict(X).argmax(axis=-1)
#     actual = y.values
#     difference = actual - predictions
#     false_positives = np.where(difference == ...)[0]
#     false_negatives = np.where(difference == ...)[0]
#     assert size < len(false_positives)
#     assert size < len(false_negatives)
#     random_sample_false_positives = np.random.choice(..., size=...)
#     random_sample_false_negatives = np.random.choice(..., size=...)
#     flattened_indices = list(
#         itertools.chain.from_iterable((random_sample_false_positives, random_sample_false_negatives))
#     )
#     tab_indices = X.iloc[flattened_indices].index.tolist()
#     return tab_indices

In [ ]:
tab_indices_train = find_incorrect_pred_indices(model=model_nn, X=x_train, y=y_train,size =1)
tab_indices_test = find_incorrect_pred_indices(model=model_nn, X=x_test, y=y_test,size = 1)

In [ ]:
explanations_train = explainer_nn.explain(tabular_data_train.to_pd().loc[tab_indices_train])
explanations_test = explainer_nn.explain(tabular_data_test.to_pd().loc[tab_indices_test])

In [ ]:
explanations_train.ipython_plot()

In [ ]:
explanations_test.ipython_plot()

# 🗝️Outro
Awesome! You made it till the end 👏 Answer the following questions to deepen your understanding.

1. Do you think the ML model prediction is similar to the orignal manual prediction? Do the predictions align well with your initial intuition?
2. We as humans have biases in our decision making. The biases might seep into the ML model as the models try to minimize the loss and improve accuracy. With the help of counterfactuals, did you see that the model has any inherent bias?
3. Do you think the counterfactuals serve a good tool to explain customers what they could do to achieve a better Risk profile? Give reasoning.

# 💰Bonus - Counterfactuals for text classification 
Let's apply counterfactuals in the context of text classification. Specifically, we will generate counter factuals for the sentiment analysis we used in Week 2 for movie reviews. We will use the same pretrained `cardiffnlp/twitter-xlm-roberta-base-sentiment` model from hugging face. The OmnixAI provides [polyjuice](https://github.com/tongshuangwu/polyjuice) explainer for counterfactuals. You can find a demonstration of `polyjuice` from Hugging Face [here](https://media1.giphy.com/media/v7yls1pusVAyo2JfPu/giphy.gif?cid=ecf05e47yq0khgmvfuggu2tukla3eavittk2oyxagnd51llz&rid=giphy.gif&ct=g)


<center><img src='https://media1.giphy.com/media/v7yls1pusVAyo2JfPu/giphy.gif?cid=ecf05e47yq0khgmvfuggu2tukla3eavittk2oyxagnd51llz&rid=giphy.gif&ct=g'></center>

In [ ]:
!pip install transformers==4.6.0 polyjuice_nlp torch omnixai

In [ ]:
# NLTk is a NLP toolkit that provides helpful lexical resources such as the wordnet (https://wordnet.princeton.edu/) which can be used to find synsets of words. Eg. car <--> automobile
import nltk
nltk.download('omw-1.4')

In [ ]:
import torch
import transformers
from polyjuice import Polyjuice
from transformers import AutoModelForSequenceClassification
from omnixai.data.text import Text
from omnixai.explainers.nlp import NLPExplainer

In [ ]:
## Before we build our transformer, lets make sure to setup the device.
## To run this notbeook via GPU: Edit -> Notebook settings -> Hardware accelerator -> GPU
## If your GPU is working, device is "cuda"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
name = "cardiffnlp/twitter-xlm-roberta-base-sentiment" 
# The pre- and post-processing functions
preprocess = lambda x: x.values
postprocess = lambda outputs: np.array([[s["score"] for s in ss] for ss in outputs])


##TODO: Build pre-trained model for sentiment analysis

# model = transformers.pipeline(
#     'sentiment-analysis',
#     model=...,
#     return_all_scores=...
# )

In [ ]:

# Build explainer using the NLPExplainer. Use "polyjuice" for explainer

# explainer = NLPExplainer(
#     explainers=[...],
#     mode="...",
#     model=...,
#     preprocess=...,
#     postprocess=...
# )


In [ ]:
## Remember our classes for sentiment analysis are as follows
model.model.config.label2id

Let us use some phrases from movie reviews. Feel free to add your own or  ask ChatGPT to generate some for you😉

In [ ]:
x = Text([
    "What a great movie!",
    "The movie had great narration and visuals despite a boring storyline"
])

In [ ]:
# Generates explanations
local_explanations = explainer.explain(x)

In [ ]:
## View explanations
local_explanations['polyjuice'].ipython_plot(index = 1)

This is the final project of the four week `Interpreting Machine Learning Models` course. We hope you had a fun learning experience. Keep engaged with the community and build it stronger💪